# Heirarchical Attention Network for text classification

In [1]:
## Uncomment command below to kill current job:
#!neuro kill $(hostname)

Our recipe is based on highly cited paper
[Hierarchical Attention Networks for Document Classification](https://arxiv.org/abs/1608.07775) (Z. Yang et al.), 
published in 2017. We classify the IMDB's reviews as positive and negative
(25k reviews for train and the same number for test). The proposed neural network architecture takes two steps:
1. It encodes sentences. The attention mechanism predicts the importance for each word in the final embedding of a sentence.
2. It encodes texts. The attention mechanism predicts the importance for each sentence in the final embedding of a text.

This architecture is exciting because we can make an illustration to understand what words and sentences were
important for prediction. You can find more information in the original article.

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from src.notebooks_utils import display_predict
from src.dataset import get_test_dataset, collate_docs, ImdbReviewsDataset
from src.model import HAN
from src.const import RESULT_DIR

### Load IMDB reviews dataset

In [3]:
dataset = get_test_dataset()
itow = dict(zip(dataset.vocab.values(), dataset.vocab.keys()))

Dataset loading from /data/aclImdb/test.


### Load your trained model

In [4]:
# path_to_ckpt = RESULT_DIR / 'logs' / 'checkpoints' / 'best.pth'  # ckpt will apear here you run training
# model = HAN.from_imbd_ckpt(path_to_ckpt)

### Load pretrained model

In [5]:
!! chmod 700 ../src/download_pretrained.sh && bash -c ../src/download_pretrained.sh

path_to_ckpt = RESULT_DIR / 'pretrained_hier.pth'
model = HAN.from_imbd_ckpt(path_to_ckpt)

Model was loaded from ../results/pretrained_hier.pth.


## Display predict for reviews from test

In [6]:
from random import randint

idx = randint(1, len(dataset))
batch = collate_docs([dataset[idx]])
display_predict(model=model, batch=batch, itow=itow)
    
print('Raw review:')
print(open(dataset._paths[idx], 'r').read())

Predict: positive (confedence: 0.875)
Ground truth: positive.


Sent #1 | seeing other people is a daring romantic comedy about a couple named ed and alice jay mohr and julianne nicholson who are engaged and plan to be wed soon

Sent #2 | they live together but are both having doubts about their relationship

Sent #3 | alice realizes she s had so few sexual relationships in the past she might just be marrying ed because she s never felt anything else

Sent #4 | so they agree to begin fooling around with other people for a while to test their own relationship

...

Sent #7 | when ed begins having sex with a college girl he begins to become addicted and almost forget about alice when he realizes this it scares him

Sent #8 | i t heard anything about this film in advance but i enjoyed it

Sent #9 | it s not extremely well made and definitely has that purposefully low budget indie feel to it but it s a lot better than most romantic comedies out there in the mainstream today

Sent #10 | check it out if you get a chance

Raw review:
"Seeing Other People" is a daring romantic comedy about a couple named Ed and Alice (Jay Mohr and Julianne Nicholson) who are engaged and plan to be wed soon. They live together but are both having doubts about their relationship. Alice realizes she's had so few sexual relationships in the past, she might just be marrying Ed because she's never felt anything else. So they agree to begin fooling around with other people for a while to test their own relationship.<br /><br />The movie balances a prescient question - by focusing too much on the "What if?" aspects of life, can it in fact do the opposite and only make you feel more constrained? When Ed begins having sex with a college girl he begins to become addicted and almost forget about Alice - when he realizes this, it scares him.<br /><br />I hadn't heard anything about this film in advance but I enjoyed it. It's not extremely well-made and definitely has that purposefully low-budget indie feel to it - but it's a lot bett

## Display predict on your own review

In [7]:
text_str = '''

I really like films like this! I discover several new talents for myself. 
The sound was great and picture as well. So, I am going to see it again.

'''


text, snt_max_len, txt_len  = dataset.tokenize_plane_text(text_str)

batch = collate_docs([{'txt': text, 'snt_len': snt_max_len,
                       'txt_len': txt_len, 'label': -1}])
batch['targets'] = None

display_predict(model=model, batch=batch, itow=itow)

Predict: positive (confedence: 0.334)


Sent #1 | i really like films like this !

Sent #2 | i discover several new talents for myself

Sent #3 | the sound was great and picture as well

Sent #4 | so i am going to see it again